# Filtering out elements based on hard criteria

In a lot of cases hard filtering is needed, when we specifically do not want the result set to contain some items, no matter how deep we scroll into the results. This can be achieved via the `.filter` clause in the `Query`.

In [1]:
%pip install superlinked==9.38.1

In [2]:
import pandas as pd

from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import Schema
from superlinked.framework.common.schema.schema_object import (
    String,
    Integer,
    StringList,
)
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

pd.set_option("display.max_colwidth", 100)

In [3]:
class Paragraph(Schema):
    id: IdField
    body: String
    author: String
    length: Integer
    tags: StringList


paragraph = Paragraph()

body_space = TextSimilaritySpace(
    text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2"
)

In [4]:
paragraph_index = Index(
    [body_space],
    fields=[paragraph.author, paragraph.body, paragraph.length, paragraph.tags],
)

> **_NOTE:_** The index definition requires the fields that we plan to create filters for.

Now let's add some data and try it out!

In [5]:
source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [6]:
source.put(
    [
        {
            "id": "paragraph-1",
            "body": "The first thing Adam wrote.",
            "author": "Adam",
            "length": 300,
            "tags": ["old", "interesting"],
        },
        {
            "id": "paragraph-2",
            "body": "The first thing Bob wrote.",
            "author": "Bob",
            "length": 500,
            "tags": ["fresh", "dull", "useful"],
        },
        {
            "id": "paragraph-3",
            "body": "The second thing Adam wrote.",
            "author": "Adam",
            "length": 400,
            "tags": ["interesting", "funny", "fresh"],
        },
    ]
)

## Using the .filter clause

### Comparisons
Provides the opportunity to write filters on the result set. For example I can ask for articles written by Adam...

In [7]:
adam_query = Query(paragraph_index).find(paragraph).filter(paragraph.author == "Adam")
adam_result = app.query(adam_query)

adam_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0
1,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


> **_NOTE:_**  As we are only using filters, `similarity_score` is always 0, as there is effectively no vector search in this case.

...or not Adam.

In [8]:
bob_query = Query(paragraph_index).find(paragraph).filter(paragraph.author != "Adam")
bob_result = app.query(bob_query)

bob_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0


It is possible to filter for being greater or smaller (and equal) to certain field.

In [9]:
greater_equal_query = (
    Query(paragraph_index).find(paragraph).filter(paragraph.length <= 400)
)
greater_equal_result = app.query(greater_equal_query)

greater_equal_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0
1,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


### Logical operations

And we can also stack multiple filters to form AND type of relationship.

In [10]:
stacked_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.author == "Adam")
    .filter(paragraph.body == "The first thing Adam wrote.")
)
stacked_result = app.query(stacked_query)

stacked_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0


It is possible to filter for being greateer or smaller (and equal) to certain field

In [11]:
or_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter((paragraph.author == "Bob").or_(paragraph.length == 300))
)
or_result = app.query(or_query)

or_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0
1,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0


... or simply using the well known ways of `__or__`.

In [12]:
or_query_ = (
    Query(paragraph_index)
    .find(paragraph)
    .filter((paragraph.author == "Bob") | (paragraph.length == 400))
)
or_result_ = app.query(or_query_)

or_result_.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0
1,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


### Set operations

We can filter for a paragraph having an author from a group of possible authors using the `in_` operator.

In [13]:
in_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.author.in_(Param("filter_list")))
)
in_result = app.query(in_query, filter_list=["Alice", "Adam", "Amon"])

in_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0
1,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


Or for a paragraph has an author that is not part of a list.

In [14]:
not_in_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.author.not_in_(Param("filter_list")))
)
not_in_result = app.query(not_in_query, filter_list=["Alice", "Adam", "Amon"])

not_in_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0


List type attributes could be filtered conditioning on containing a value.

In [15]:
contains_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.tags.contains(Param("filter_tag")))
)
contains_result = app.query(contains_query, filter_tag=["fresh"])

contains_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0
1,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


Contains with a `list` input work as the industry practice, returns entities containing either of the listed elements.

In [16]:
contains_multiple_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.tags.contains(Param("filter_tag")))
)
contains_result = app.query(
    contains_multiple_query, filter_tag=["fresh", "interesting"]
)

contains_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0
1,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0
2,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


To filter for entities containing all of the listed elements, stack filters.

In [17]:
contains_multiple_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.tags.contains(Param("filter_tag_1")))
    .filter(paragraph.tags.contains(Param("filter_tag_2")))
)
contains_result = app.query(
    contains_multiple_query, filter_tag_1=["fresh"], filter_tag_2=["useful"]
)

contains_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Bob wrote.,Bob,500,"[fresh, dull, useful]",paragraph-2,0.0


The same can be done using not_contains, too. With a single...

In [18]:
not_contains_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.tags.not_contains(Param("filter_tag")))
)
not_contains_result = app.query(not_contains_query, filter_tag=["fresh"])

not_contains_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The first thing Adam wrote.,Adam,300,"[old, interesting]",paragraph-1,0.0


... or with multiple values.

In [19]:
not_contains_multiple_query = (
    Query(paragraph_index)
    .find(paragraph)
    .filter(paragraph.tags.not_contains(Param("filter_tag")))
)
not_contains_multiple_result = app.query(
    not_contains_multiple_query, filter_tag=["old", "dull"]
)

not_contains_multiple_result.to_pandas()

,body,author,length,tags,id,similarity_score
0,The second thing Adam wrote.,Adam,400,"[interesting, funny, fresh]",paragraph-3,0.0


## Summary

We are supporting

#### Comparison operations
* the `==` and `!=` operators,
* the `<`, `>`, `<=` and `>=` operators,
#### Logical operations
* creating `AND` relationships by stacking filters,
* and using the `.or_` or simply `|` to create `OR` relationships.
#### Set operations
* the `.in_` and `.not_in_` operator test String fields having a value from a collection,
* the `.contains` and `not_contains_` operator tests StringList containing either of the values from a collection,